In [ ]:
!git clone https://github.com/CAMMA-public/SurgVLP.git

In [2]:
%cd /kaggle/working/SurgVLP

/kaggle/working/SurgVLP


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip install git+https://github.com/CAMMA-public/SurgVLP.git

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from tqdm import tqdm
import surgvlp
from mmengine.config import Config
import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

class EndoscopeClassifier(nn.Module):
    def __init__(self, peskaVLP_model, num_classes=7, freeze_encoder=True):
        super().__init__()
        # Store the whole PeskaVLP model to use its image encoding functionality
        self.peskaVLP = peskaVLP_model
        
        # Freeze the PeskaVLP model if specified
        if freeze_encoder:
            for param in self.peskaVLP.parameters():
                param.requires_grad = False
                
        # Get the embedding dimension from the model
        embedding_dim = 768
        
        # Add classification head
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        # Use the entire PeskaVLP model to get image features
        with torch.no_grad():
            features = self.peskaVLP(x, None, mode='video')['img_emb']

            # Ensure we have the right shape
            if isinstance(features, tuple):
                features = features[0]
            if features.dim() > 2:
                features = features[:, 0]  # Get the [CLS] token features
                
        # Pass through classification head
        return self.classifier(features)

class ENTJsonDataset(Dataset):
    """Dataset backed by the Train json annotation list."""

    LABELS = ['nose-right', 'nose-left', 'ear-right', 'ear-left', 'vc-open', 'vc-closed', 'throat']

    def __init__(self, items, img_dir, transform=None):
        self.items = items  # list of dicts from json
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_idx = {c: i for i, c in enumerate(self.LABELS)}

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]
        img_name = item['Path']
        label_str = item['Classification']
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(self.label_to_idx[label_str], dtype=torch.long)

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    all_targets = []
    all_preds = []
    
    progress_bar = tqdm(dataloader, desc='Training')
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # accumulate for balanced accuracy
        all_targets.extend(labels.cpu().tolist())
        all_preds.extend(predicted.cpu().tolist())
        
        progress_bar.set_postfix({
            'loss': total_loss/len(dataloader),
            'acc': 100.*correct/total
        })
        
    bal_acc = balanced_accuracy_score(all_targets, all_preds) * 100.0
    return total_loss/len(dataloader), 100.*correct/total, bal_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_targets = []
    all_preds = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # accumulate for balanced accuracy
            all_targets.extend(labels.cpu().tolist())
            all_preds.extend(predicted.cpu().tolist())
            
    bal_acc = balanced_accuracy_score(all_targets, all_preds) * 100.0
    return total_loss/len(dataloader), 100.*correct/total, bal_acc

# ------------------------------------------------------------------
# Loss: FocalLoss implementation
# ------------------------------------------------------------------

class FocalLoss(nn.Module):
    """Focal Loss for multi-class classification.

    Args:
        alpha (float): balancing factor for positive/negative examples.
        gamma (float): focusing parameter that reduces the relative loss
                       for well-classified examples.
        reduction (str): "none", "mean", or "sum".
    """

    def __init__(self, alpha: float = 1.0, gamma: float = 2.0, reduction: str = 'mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor):
        # Compute standard cross-entropy loss (per sample)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)  # prevents nans when probability 0
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load PeskaVLP model
    configs = Config.fromfile('./tests/config_peskavlp.py')['config']
    peskaVLP_model, preprocess = surgvlp.load(configs.model_config, device=device)
    
    # Create our classifier
    model = EndoscopeClassifier(peskaVLP_model, num_classes=7, freeze_encoder=True)
    model = model.to(device)

    # Dataset and DataLoader
    train_transform = preprocess
    val_transform   = preprocess
    
    # ------------------------------------------------------------------
    # Prepare ENT datasets
    train_json_path = Path('/kaggle/input/augmented-data/cls_train.json')
    train_img_dir   = '/kaggle/input/augmented-data/images'

    test_json_path  = Path('/kaggle/input/datatrain/cls_test.json')
    test_img_dir    = '/kaggle/input/datatrain/Train1/Train1/images'

    if not train_json_path.exists():
        raise FileNotFoundError(f'Missing training labels: {train_json_path}')
    if not test_json_path.exists():
        raise FileNotFoundError(f'Missing test labels: {test_json_path}')

    with open(train_json_path, 'r', encoding='utf-8') as f:
        train_mapping = json.load(f)
    with open(test_json_path, 'r', encoding='utf-8') as f:
        test_mapping = json.load(f)

    train_records = [{'Path': fn, 'Classification': lbl} for fn, lbl in train_mapping.items()]
    test_records  = [{'Path': fn, 'Classification': lbl} for fn, lbl in test_mapping.items()]

    # Further split train_records to obtain a validation set (10% of train)
    train_records, val_records = train_test_split(
        train_records,
        test_size=0.2,
        random_state=42,
        stratify=[r['Classification'] for r in train_records]
    )

    train_dataset = ENTJsonDataset(train_records, train_img_dir, transform=train_transform)
    val_dataset   = ENTJsonDataset(val_records,   train_img_dir, transform=val_transform)
    test_dataset  = ENTJsonDataset(test_records,  test_img_dir, transform=val_transform)

    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=16
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=16
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=16
    )
    
    # Loss and optimizer
    criterion = FocalLoss(alpha=1.0, gamma=2.0, reduction='mean')
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=2, verbose=True
    )

    # ------------------------------------------------------------------
    # Resume from checkpoint if available
    # ------------------------------------------------------------------
    ckpt_path = ''
    start_epoch = 0
    best_val_acc = 0
    if Path(ckpt_path).is_file():
        ckpt = torch.load(ckpt_path, map_location=device)
        model.load_state_dict(ckpt['model_state_dict'])
        try:
            optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        except ValueError:
            print('Optimizer state mismatch, continuing with fresh optimizer')
        best_val_acc = ckpt.get('val_accuracy', 0)
        start_epoch = ckpt.get('epoch', 0) + 1
        print(f"Resumed from checkpoint {ckpt_path} (epoch {start_epoch}, best val acc {best_val_acc:.2f}%)")

    # Training loop – continue up to total_epochs
    total_epochs = 300  # change as needed
    
    for epoch in range(start_epoch, total_epochs):
        print(f"\nEpoch {epoch+1}/{total_epochs}")
        
        # Training phase
        train_loss, train_acc, train_bal_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device
        )
        print(f"Training - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%, BalAcc: {train_bal_acc:.2f}%")
        
        # Validation phase
        val_loss, val_acc, val_bal_acc = validate(model, val_loader, criterion, device)
        print(f"Validation - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%, BalAcc: {val_bal_acc:.2f}%")
        
        # Update learning rate based on validation loss
        scheduler.step(val_loss)
        
        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': best_val_acc,
                'train_accuracy': train_acc,
            }, '/kaggle/working/checkpoint.pth')
            print(f"Saved new best model with validation accuracy: {val_acc:.2f}%")

    # ------------------------------------------------------------------
    # Evaluate the best checkpoint on the held-out test set
    # ------------------------------------------------------------------
    if Path('/kaggle/working/checkpoint.pth').is_file():
        best_ckpt = torch.load('/kaggle/working/checkpoint.pth', map_location=device)
        model.load_state_dict(best_ckpt['model_state_dict'])
        print("Loaded best checkpoint for testing.")

    test_loss, test_acc, test_bal_acc = validate(model, test_loader, criterion, device)
    print(f"Test - Loss: {test_loss:.4f}, Acc: {test_acc:.2f}%, BalAcc: {test_bal_acc:.2f}%")

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 199MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when 

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Training - Loss: 0.0636, Acc: 90.99%, BalAcc: 92.53%



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Validation - Loss: 0.1061, Acc: 86.78%, BalAcc: 88.68%
Loaded best checkpoint for testing.


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test - Loss: 0.1982, Acc: 77.96%, BalAcc: 80.11%
